In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import re
import pycountry
import openpyxl
import random

random.seed(42)

ocde = [
    'Australia',
    'Austria',
    'Belgium',
    'Canada',
    'Chile',
    'Czechia',
    'Denmark',
    'Estonia',
    'Finland',
    'France',
    'Germany',
    'Greece',
    'Hungary',
    'Iceland',
    'Ireland',
    'Israel',
    'Italy',
    'Japan',
    'South Korea',
    'Luxembourg',
    'Mexico',
    'Netherlands',
    'New Zealand',
    'Norway',
    'Poland',
    'Portugal',
    'Slovakia',
    'Slovenia',
    'Spain',
    'Sweden',
    'Switzerland',
    'Türkiye',
    'United Kingdom',
    'United States'
]


df = pd.read_json('../../data/prokanon.mt_combs4_newrel_newage.json')
df.loc[(df['mau'] == 1000), 'mau'] = df['dau']
df['mau'] = df['mau'].apply(lambda x: x if x >= 109 else random.randint(1, 109))
df.sort_values(by='mau',inplace=True)


df['cumsum'] = df.groupby('country')['mau'].cumsum()

sum_per_country = df.groupby('country')['mau'].sum()

df['cdf'] = df['cumsum'] / df['country'].map(sum_per_country)
paises = df[['country']].drop_duplicates()


In [3]:

res_geq090 = df[df['cdf'] >= 0.1]

idx_min = res_geq090.groupby('country')['cdf'].apply(lambda x: (x - 0.1).abs().idxmin())

results_90 = res_geq090.loc[idx_min]



res_geq095 = df[df['cdf'] >= 0.05]

idx_min = res_geq095.groupby('country')['cdf'].apply(lambda x: (x - 0.05).abs().idxmin())

results_95 = res_geq095.loc[idx_min]



res_geq099 = df[df['cdf'] >= 0.01]

idx_min = res_geq099.groupby('country')['cdf'].apply(lambda x: (x - 0.01).abs().idxmin())

results_99 = res_geq099.loc[idx_min]





for index, row in results_90[['country','mau']].iterrows():
    mask = paises['country'] == row['country']
    paises.loc[mask,'K^4_90'] = row['mau']


for index, row in results_95[['country','mau']].iterrows():
    mask = paises['country'] == row['country']
    paises.loc[mask,'K^4_95'] = row['mau']


for index, row in results_99[['country','mau']].iterrows():
    mask = paises['country'] == row['country']
    paises.loc[mask,'K^4_99'] = row['mau']



for index, value in sum_per_country.items():
    mask = paises['country'] == index
    paises.loc[mask,'RK^4_90'] = (paises.loc[mask,'K^4_90']/value) * 100 
    paises.loc[mask,'RK^4_95'] = (paises.loc[mask,'K^4_95']/value) * 100 
    paises.loc[mask,'RK^4_99'] = (paises.loc[mask,'K^4_99']/value) * 100 

print(len(df.index))

9200


In [4]:
paises = paises.sort_values(by='country',ascending=True).dropna()
paises['K^4_90'] = paises['K^4_90'].astype('int')
paises['K^4_95'] = paises['K^4_95'].astype('int')
paises['K^4_99'] = paises['K^4_99'].astype('int')
paises['RK^4_90'] = paises['RK^4_90'].round(3)
paises['RK^4_95'] = paises['RK^4_95'].round(3)
paises['RK^4_99'] = paises['RK^4_99'].round(3)



paises

paises.to_parquet('../../data/analyzed/fb/FB_newrel_newage_K^4_X_by_country.parquet')

In [5]:
df['rel_status'].drop_duplicates()

325     4
5484    6
6478    1
7361    2
6237    3
Name: rel_status, dtype: int64

In [6]:
paises

,country,K^4_90,K^4_95,K^4_99,RK^4_90,RK^4_95,RK^4_99
325,AD,845,664,104,1.903,1.496,0.234
2715,AE,206600,69900,17700,2.013,0.681,0.172
2238,AF,79700,35300,5900,1.903,0.843,0.141
4991,AG,989,798,332,1.518,1.225,0.509
275,AL,16700,8300,2000,1.396,0.694,0.167
...,...,...,...,...,...,...,...
4116,YE,95600,23900,7800,2.376,0.594,0.194
8140,YT,832,749,497,1.235,1.112,0.738
8870,ZA,470500,334400,60400,1.810,1.287,0.232
9155,ZM,52600,31000,6800,1.458,0.860,0.189
